In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 201902

## Import Data

In [2]:
fhvhv_201902 = pd.read_parquet('../fhvhv_tripdata_2019-02.parquet', engine='pyarrow')
fhvhv_201902.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02867,B02867,2019-02-01 00:01:26,2019-02-01 00:02:55,2019-02-01 00:05:18,2019-02-01 00:14:57,245,251,2.45,...,0.83,0.0,None,0.0,7.48,Y,N,N,N,None
1,HV0003,B02879,B02879,2019-02-01 00:26:08,2019-02-01 00:41:29,2019-02-01 00:41:29,2019-02-01 00:49:39,216,197,1.71,...,0.70,0.0,None,2.0,7.93,N,N,N,N,None
2,HV0005,B02510,None,2019-02-01 00:48:58,NaT,2019-02-01 00:51:34,2019-02-01 01:28:29,261,234,5.01,...,3.99,0.0,None,0.0,35.97,N,Y,N,N,None
3,HV0005,B02510,None,2019-02-01 00:02:15,NaT,2019-02-01 00:03:51,2019-02-01 00:07:16,87,87,0.34,...,0.64,0.0,None,3.0,5.39,N,Y,N,N,None
4,HV0005,B02510,None,2019-02-01 00:06:17,NaT,2019-02-01 00:09:44,2019-02-01 00:39:56,87,198,6.84,...,2.16,0.0,None,4.0,17.07,N,Y,N,N,None


In [3]:
fhvhv_201902.shape

(20159102, 24)

查看每種license_num各有多少record

In [4]:
print(fhvhv_201902['hvfhs_license_num'].value_counts())
print(fhvhv_201902['hvfhs_license_num'].value_counts(normalize=True))

hvfhs_license_num
HV0003    13504994
HV0005     4690916
HV0004      983926
HV0002      979266
Name: count, dtype: int64
hvfhs_license_num
HV0003    0.669920
HV0005    0.232695
HV0004    0.048808
HV0002    0.048577
Name: proportion, dtype: float64


## Uber (HV0003)

僅繼續分析Uber (HV0003)資料

In [5]:
uber_201902 = fhvhv_201902[fhvhv_201902['hvfhs_license_num'] == 'HV0003']
uber_201902.shape

(13504994, 24)

共有24個columns，分兩部分查看column info

In [6]:
uber_201902.iloc[:, :12].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 13504994 entries, 0 to 20159101
Data columns (total 12 columns):
 #   Column                Non-Null Count     Dtype         
---  ------                --------------     -----         
 0   hvfhs_license_num     13504994 non-null  object        
 1   dispatching_base_num  13504589 non-null  object        
 2   originating_base_num  13504589 non-null  object        
 3   request_datetime      13504994 non-null  datetime64[ns]
 4   on_scene_datetime     13504994 non-null  datetime64[ns]
 5   pickup_datetime       13504994 non-null  datetime64[ns]
 6   dropoff_datetime      13504994 non-null  datetime64[ns]
 7   PULocationID          13504994 non-null  int64         
 8   DOLocationID          13504994 non-null  int64         
 9   trip_miles            13504994 non-null  float64       
 10  trip_time             13504994 non-null  int64         
 11  base_passenger_fare   13504994 non-null  float64       
dtypes: datetime64[ns](4), float64(2

In [7]:
uber_201902.iloc[:, 12:].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 13504994 entries, 0 to 20159101
Data columns (total 12 columns):
 #   Column                Non-Null Count     Dtype  
---  ------                --------------     -----  
 0   tolls                 13504994 non-null  float64
 1   bcf                   13504994 non-null  float64
 2   sales_tax             13504994 non-null  float64
 3   congestion_surcharge  13504994 non-null  float64
 4   airport_fee           0 non-null         object 
 5   tips                  13504994 non-null  float64
 6   driver_pay            13504994 non-null  float64
 7   shared_request_flag   13504994 non-null  object 
 8   shared_match_flag     13504994 non-null  object 
 9   access_a_ride_flag    13504994 non-null  object 
 10  wav_request_flag      13504994 non-null  object 
 11  wav_match_flag        0 non-null         object 
dtypes: float64(6), object(6)
memory usage: 1.3+ GB


## EDA

## 計算總車資

先確認有沒有base_passenger_fare為0的紀錄

In [8]:
uber_base_fare_0 = uber_201902[uber_201902['base_passenger_fare'] == 0]
uber_base_fare_0.shape

(12467, 24)

檢查車資為0的紀錄其他column

In [14]:
uber_base_fare_0.loc[:, ['trip_miles', 'trip_time', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips']].head()

,trip_miles,trip_time,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips
34,1.50,315,0.0,0.0,0.0,0.0,None,0.0
5970,0.90,303,0.0,0.0,0.0,0.0,None,0.0
6103,0.24,150,0.0,0.0,0.0,0.0,None,0.0
6620,0.59,123,0.0,0.0,0.0,0.0,None,0.0
11651,0.21,191,0.0,0.0,0.0,0.0,None,0.0


In [22]:
# print(f'trip_time_max: {uber_base_fare_0["trip_time"].describe()}')
# print(f'tolls: {uber_base_fare_0["tolls"].describe()}')
# print(f'bcf: {uber_base_fare_0["bcf"].describe()}')
# print(f'sales_tax: {uber_base_fare_0["sales_tax"].describe()}')
# print(f'congestion_surcharge: {uber_base_fare_0["congestion_surcharge"].describe()}')
print(f'tips: {uber_base_fare_0["tips"].describe()}')

tips: count    12467.000000
mean         0.053814
std          0.364252
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         10.000000
Name: tips, dtype: float64


為什麼這些車資會是0? 折扣?

不包含driver_pay，driver_pay為該趟行程Uber支付給司機的費用

In [9]:
# uber_201902.total_amount = uber_201902[['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips']].sum(axis=1)
# uber_201902.total_amount

C:\Users\User\AppData\Local\Temp\ipykernel_7360\2361652500.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  uber_201902.total_amount = uber_201902[['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips']].sum(axis=1)


0           10.41
1           10.81
8            7.78
9            7.86
10          11.54
            ...  
20159096    17.28
20159098    30.52
20159099     9.16
20159100    25.61
20159101    15.36
Length: 13504994, dtype: object

# 202304

## Import Data

In [10]:
fhvhv_202304 = pd.read_parquet('../fhvhv_tripdata_2023-04.parquet', engine='pyarrow')
fhvhv_202304.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2023-04-01 00:01:28,2023-04-01 00:02:41,2023-04-01 00:03:03,2023-04-01 00:15:42,174,126,5.10,...,1.82,0.00,0.0,0.00,13.83,N,N,,N,N
1,HV0003,B03404,B03404,2023-04-01 00:39:40,2023-04-01 00:41:20,2023-04-01 00:43:42,2023-04-01 01:01:08,211,163,3.11,...,3.56,2.75,0.0,9.52,24.84,N,N,,N,N
2,HV0003,B03404,B03404,2023-03-31 23:56:31,2023-04-01 00:00:20,2023-04-01 00:01:01,2023-04-01 00:09:48,222,76,2.40,...,1.08,0.00,0.0,0.00,8.10,Y,N,,N,N
3,HV0003,B03404,B03404,2023-04-01 00:12:35,2023-04-01 00:16:04,2023-04-01 00:17:17,2023-04-01 00:19:12,76,124,0.43,...,0.70,0.00,0.0,0.00,5.40,N,N,,N,N
4,HV0003,B03404,B03404,2023-04-01 00:46:23,2023-04-01 00:47:30,2023-04-01 00:47:49,2023-04-01 01:05:23,263,247,4.18,...,1.48,2.75,0.0,0.00,15.65,N,N,,N,N


## Uber (HV0003)

僅繼續分析Uber (HV0003)資料

In [11]:
fhvhv_202304.shape

(19144903, 24)

In [12]:
uber_202304 = fhvhv_201902[fhvhv_201902['hvfhs_license_num'] == 'HV0003']
uber_202304.shape

(13504994, 24)

稅的計算是根據哪些費用?

FHV，看星期幾的搭乘次數、小費最多  
行程距離與小費畫成散點圖